In [25]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List, ClassVar, Dict
from notion_client import Client
import os
import openai
import json
import re
load_dotenv(override=True)

# Fetch the Notion API key
notion_api_key = os.getenv("NOTION_API_KEY")
notion_parent_id = os.getenv("NOTION_PARENT_ID")

youtube_api_key = os.getenv("GOOGLE_API_KEY")

# Initialize the Notion client
notion = Client(auth=notion_api_key)


In [26]:
topics = [{"topic": "SIRD Modelling", "page_id": "1d36ccbb-ecba-8106-9696-c0f516f75632", "content": "[[Introduction]]\nSIRD modeling is a mathematical framework used to understand the spread of infectious diseases within a population. It extends the basic SIR model by incorporating a compartment for deceased individuals. This model helps public health officials predict the course of an epidemic and evaluate the impact of interventions.\n\n[[Key Definitions]]\n- **Susceptible (S)**: The portion of the population that is not yet infected with the disease but is at risk of becoming infected.\n- **Infectious (I)**: The group of individuals who have been infected and are capable of spreading the disease to susceptible individuals.\n- **Recovered (R)**: Individuals who have recovered from the disease and are assumed to have gained immunity, thus no longer susceptible.\n- **Deceased (D)**: Individuals who have died from the disease, representing the fatality aspect of the epidemic.\n\n[[Relevant Formulas]]\n- **Rate of Change of Susceptible Individuals**\n\\( \\frac{dS}{dt} = -\\beta \\frac{SI}{N} \\)\n\n- **Rate of Change of Infectious Individuals**\n\\( \\frac{dI}{dt} = \\beta \\frac{SI}{N} - \\gamma I - \\mu I \\)\n\n- **Rate of Change of Recovered Individuals**\n\\( \\frac{dR}{dt} = \\gamma I \\)\n\n- **Rate of Change of Deceased Individuals**\n\\( \\frac{dD}{dt} = \\mu I \\)\n\nWhere \\( \\beta \\) is the transmission rate, \\( \\gamma \\) is the recovery rate, \\( \\mu \\) is the mortality rate, and \\( N \\) is the total population.\n\n[[Examples]]\n1. **Influenza Outbreak**: Consider a city experiencing an influenza outbreak. The SIRD model can help predict how quickly the disease will spread, how many people will recover, and the potential fatalities. By adjusting parameters like the transmission rate \\( \\beta \\), public health officials can simulate the effects of interventions such as vaccination or social distancing.\n\n2. **COVID-19 Pandemic**: During the COVID-19 pandemic, SIRD models were used extensively to forecast the progression of the virus. By inputting real-time data, these models helped governments decide when to implement lockdowns or increase healthcare resources, aiming to minimize the number of infections and deaths.\n\n[[Reflective Questions]]\n1. How does the inclusion of the deceased compartment in the SIRD model provide a more comprehensive view of an epidemic compared to the SIR model?\n2. What factors could influence the transmission rate \\( \\beta \\) in a real-world scenario?\n3. Why is it important to consider the recovery rate \\( \\gamma \\) when modeling infectious diseases?\n4. How can public health interventions alter the parameters of the SIRD model to control an outbreak?\n5. In what ways might the assumptions of the SIRD model limit its applicability to real-world epidemics?"}, {"topic": "Ordinary Differential Equations (ODEs)", "page_id": "1d36ccbb-ecba-81ed-97bb-f19b23dd392b", "content": "[[Introduction]]\nOrdinary Differential Equations (ODEs) are equations involving functions and their derivatives. They are fundamental in describing various physical phenomena, such as motion, growth, and decay processes. Understanding ODEs is crucial for modeling and solving problems in engineering, physics, biology, and economics.\n\n[[Key Definitions]]\n- **Ordinary Differential Equation (ODE):** An equation involving a function of one independent variable and its derivatives.\n- **Order of an ODE:** The highest derivative present in the equation. For example, in \\( \\frac{d^2y}{dx^2} + 3\\frac{dy}{dx} + 2y = 0 \\), the order is 2.\n- **Linear ODE:** An ODE in which the dependent variable and its derivatives appear linearly. For example, \\( \\frac{dy}{dx} + p(x)y = g(x) \\) is a linear first-order ODE.\n\n[[Relevant Formulas]]\n**First-Order Linear ODE:**\n\\( \\frac{dy}{dx} + p(x)y = g(x) \\)\n\n**Second-Order Linear Homogeneous ODE:**\n\\( a\\frac{d^2y}{dx^2} + b\\frac{dy}{dx} + cy = 0 \\)\n\n**General Solution of First-Order Linear ODE:**\n\\( y(x) = e^{-\\int p(x) \\, dx} \\left( \\int e^{\\int p(x) \\, dx} g(x) \\, dx + C \\right) \\)\n\n[[Examples]]\n1. **Population Growth:** The rate of change of a population \\( P \\) over time \\( t \\) can be modeled by the ODE \\( \\frac{dP}{dt} = rP \\), where \\( r \\) is the growth rate. This represents exponential growth, common in unrestricted environments.\n\n2. **Cooling of an Object:** Newton's Law of Cooling states that the rate of change of temperature \\( T \\) of an object is proportional to the difference between its temperature and the ambient temperature \\( T_a \\). This can be expressed as \\( \\frac{dT}{dt} = -k(T - T_a) \\), where \\( k \\) is a positive constant.\n\n[[Reflective Questions]]\n1. What distinguishes a linear ODE from a nonlinear ODE?\n2. How does the order of an ODE affect the complexity of its solutions?\n3. Why is it important to understand the initial conditions when solving an ODE?\n4. How can ODEs be used to model real-world phenomena such as electrical circuits or mechanical vibrations?\n5. What methods can be used to solve a second-order linear homogeneous ODE?"}, {"topic": "Least Squares Regression", "page_id": "1d36ccbb-ecba-81d2-ba42-dd654dab265b", "content": "[[Introduction]]\nLeast squares regression is a statistical method used to determine the best-fitting line through a set of data points by minimizing the sum of the squares of the vertical distances of the points from the line. This technique is widely used in predictive modeling and data analysis to understand relationships between variables.\n\n[[Key Definitions]]\n- **Least Squares Method**: A mathematical approach used to find the line of best fit by minimizing the sum of the squares of the residuals (the differences between observed and predicted values).\n- **Residual**: The difference between an observed value and the value predicted by a model. In the context of least squares, it is the vertical distance from a data point to the regression line.\n- **Regression Line**: A line that best describes the relationship between the independent variable \\( x \\) and the dependent variable \\( y \\) in a dataset.\n\n[[Relevant Formulas]]\n- **Equation of the Regression Line**\n\\( y = mx + b \\)\nwhere \\( m \\) is the slope and \\( b \\) is the y-intercept.\n\n- **Slope of the Regression Line**\n\\( m = \\frac{n(\\sum xy) - (\\sum x)(\\sum y)}{n(\\sum x^2) - (\\sum x)^2} \\)\n\n- **Y-intercept of the Regression Line**\n\\( b = \\frac{(\\sum y)(\\sum x^2) - (\\sum x)(\\sum xy)}{n(\\sum x^2) - (\\sum x)^2} \\)\n\n[[Examples]]\n- **Example 1: Predicting House Prices**\nSuppose a real estate analyst wants to predict house prices based on square footage. By collecting data on various houses, the analyst can use least squares regression to find the line of best fit, which will help predict the price of a house given its size.\n\n- **Example 2: Estimating Fuel Efficiency**\nAn automotive engineer is interested in understanding the relationship between a car's weight and its fuel efficiency. By plotting the weight of several cars against their miles per gallon (MPG), the engineer can apply least squares regression to estimate how changes in weight might affect fuel efficiency.\n\n[[Reflective Questions]]\n1. What are the main assumptions behind the least squares regression method?\n2. How does the least squares method minimize the error in predictions?\n3. In what scenarios might least squares regression not be the best method to use?\n4. How can outliers affect the results of a least squares regression analysis?\n5. What is the significance of the slope and intercept in the context of a regression line?"}, {"topic": "Paper Summary", "page_id": "1d36ccbb-ecba-812d-ba68-e08bea7f7274", "content": "summary of paper"}]

In [32]:
class QuizToolSchema(BaseModel): 

    """Input Schema for QuizTool."""
    topics: List[Dict[str, str]] = Field(description="The list of topic pages, their IDs, and their page contents.")

class QuizTool(Tool[None]):

    """ Quiz Tool for creating quizzes for topics and creating separate pages for the quizzes. 
        It will base the quizzes on the lessons and topics provided.
    """

    id: ClassVar[str] = "Quiz_tool"
    name: ClassVar[str] = "Quiz Tool"
    description: ClassVar[str] = "A tool that creates quizzes for topics and creates separate pages for the quizzes." 
    args_schema = QuizToolSchema
    output_schema: ClassVar[tuple[str, str]] = (
        "str",
        "Confirmation of Quiz Creation"
    )


    def run(self, context: ToolRunContext, topics: List[Dict[str, str]]) -> str: 
        pass

    def create_quiz(self, topic: dict) -> List[Dict[any, str]]: 
        """Creates a quiz for the given topic and returns the quiz URL."""
        client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        quiz = {}
        quiz["topic"] = topic["topic"]

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", 
                 "content": (
                        "You are a tutor who creates quizzes for students on given lessons. "
                        "All math must be written using inline LaTeX formatted as \\( ... \\). "
                        "Do NOT use display math like \\[...\\], $$...$$, or {{math: ...}}. "
                        "Do NOT wrap equations in square brackets [ ... ] or parentheses like (\\( ... \\)). "
                        "Only use valid LaTeX commands (e.g., \\frac, \\int, \\sum, \\left(, \\right)). "
                        "Each Question should be a multiple choice question with 4 options. "
                        "Output each question in the following format: "
                        "{\"question\": \"Question text\", \"options\": [\"Option A\", \"Option B\", \"Option C\", \"Option D\"], \"answer\": \"Correct answer (one of A, B, C, D)\"}."
                            )},
                {"role": "user", "content": (
                    f"Write a comprehensive multiple choice quiz on this lesson: {topic['content']} on this topic: {topic['topic']}.\n"
                )}
            ],
            temperature=0.5
        )
        
        data = response.choices[0].message.content
        escaped_data = data.replace('\\', '\\\\')
        pattern = r'\{.*?"question":.*?"options":\s*\[.*?\].*?"answer":\s*".*?"\}'
        # Find all JSON-like strings
        matches = re.findall(pattern, escaped_data, re.DOTALL) 
        quiz = [json.loads(match) for match in matches]
        print(quiz) 
        # if response.choices and response.choices[0].message.content:
        #     quiz = response.choices[0].message.content
        # else:
        #     quiz = "Failed to generate questions."

        # for q in quiz:
        #     print(f"Q: {q['question']}")
        #     for i, opt in enumerate(q['options']):
        #         print(f"  {chr(65+i)}. {opt}")
        #     print(f"Answer: {q['answer']}")
        
        return quiz
    
    def create_quiz_page(self, topic: dict, quiz: List[Dict[str, str]]) -> None:

        # Create the Quiz page under the topic['page_id']
        page_data = {
            "parent": {"page_id": topic["page_id"]},
            "properties": {
                "title": [
                    {
                        "type": "text",
                        "text": {"content": "Quiz"}
                    }
                ]
            },
            "children": []
        }

        # Add each question with its options
        for q in quiz:
            # Add question
            page_data["children"].append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text", "text": {"content": f"❓ {q['question']}"}}]
                }
            })

            # Add options
            for i, opt in enumerate(q['options']):
                letter = chr(65 + i)
                is_answer = (letter == q["answer"])
                prefix = "✅" if is_answer else f"   {letter}."
                page_data["children"].append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [{"type": "text", "text": {"content": f"{prefix} {opt}"}}]
                    }
                })

            # Add spacing
            page_data["children"].append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {"rich_text": [{"type": "text", "text": {"content": ""}}]}
            })

        # Create the page
        response = requests.post(notion_api_url, headers=headers, json=page_data)

        if response.status_code == 200:
            print("Quiz page created successfully!")
            return response.json()
        else:
            print(f"Error: {response.status_code}, {response.text}")
            return None


In [33]:
tool = QuizTool()
tool.create_quiz(topics[0])

[{'question': 'What is the primary addition in the SIRD model compared to the SIR model?', 'options': ['A new compartment for susceptible individuals', 'A new compartment for deceased individuals', 'A new compartment for infectious individuals', 'A new compartment for recovered individuals'], 'answer': 'B'}, {'question': 'Which of the following represents the rate of change of susceptible individuals in the SIRD model?', 'options': ['\\( \\frac{dS}{dt} = \\beta \\frac{SI}{N} \\)', '\\( \\frac{dS}{dt} = -\\beta \\frac{SI}{N} \\)', '\\( \\frac{dS}{dt} = \\gamma I \\)', '\\( \\frac{dS}{dt} = \\mu I \\)'], 'answer': 'B'}, {'question': 'In the SIRD model, what does the parameter \\( \\mu \\) represent?', 'options': ['The transmission rate', 'The recovery rate', 'The mortality rate', 'The total population'], 'answer': 'C'}, {'question': 'Which equation represents the rate of change of infectious individuals in the SIRD model?', 'options': ['\\( \\frac{dI}{dt} = \\gamma I \\)', '\\( \\frac{dI

[{'question': 'What is the primary addition in the SIRD model compared to the SIR model?',
  'options': ['A new compartment for susceptible individuals',
   'A new compartment for deceased individuals',
   'A new compartment for infectious individuals',
   'A new compartment for recovered individuals'],
  'answer': 'B'},
 {'question': 'Which of the following represents the rate of change of susceptible individuals in the SIRD model?',
  'options': ['\\( \\frac{dS}{dt} = \\beta \\frac{SI}{N} \\)',
   '\\( \\frac{dS}{dt} = -\\beta \\frac{SI}{N} \\)',
   '\\( \\frac{dS}{dt} = \\gamma I \\)',
   '\\( \\frac{dS}{dt} = \\mu I \\)'],
  'answer': 'B'},
 {'question': 'In the SIRD model, what does the parameter \\( \\mu \\) represent?',
  'options': ['The transmission rate',
   'The recovery rate',
   'The mortality rate',
   'The total population'],
  'answer': 'C'},
 {'question': 'Which equation represents the rate of change of infectious individuals in the SIRD model?',
  'options': ['\\( \\f